In [50]:
import numpy as np
import cv2
from tqdm import tqdm
import os
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import time
from torch.optim import Adam
from sklearn.metrics import precision_recall_fscore_support
import pickle

In [51]:
train_dir = 'pictures'
test_dir = 'test'

In [52]:
def label_img(img):
    img_label = img.split('.')[0]
    return img_label

In [53]:
def create_train_data():
    imgs = np.empty((20000,300,20,3))
    labels = []
    counter = 0
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img)
        path = os.path.join(train_dir,img)
        img = cv2.imread(path)
        img = np.array(img)/255
        img = apply_pca(img)
        imgs[counter] = img
        labels.append(label)
        counter+=1
    return imgs,labels

In [54]:
def create_test_data():
    imgs = np.empty((20000,300,20,3))
    labels = []
    counter = 0
    for img in tqdm(os.listdir(test_dir)):
        label = label_img(img)
        path = os.path.join(test_dir,img)
        img = cv2.imread(path)
        img = np.array(img)/255
        img = apply_pca(img)
        imgs[counter] = img
        labels.append(label)
        counter+=1
    return imgs,labels

In [55]:
def apply_pca(img):
    pca = PCA(20)
    #apply pca
    blue= np.squeeze(img[:,:,0])
    green = np.squeeze(img[:,:,1])
    red = np.squeeze(img[:,:,2])
    red_transformed = pca.fit_transform(red)
    #Applying to Green channel and then applying inverse transform to transformed array.
    green_transformed = pca.fit_transform(green)
    #Applying to Blue channel and then applying inverse transform to transformed array.
    blue_transformed = pca.fit_transform(blue)
    img = np.stack((red_transformed, green_transformed, blue_transformed), axis = 2)
    return img

In [56]:
# def apply_pca(x):
#     pca = PCA(50)
#     pca_x = [None] * len(x)
#     for i in tqdm(range(len(x))):
#         #apply pca
#         img = x[i]
#         blue= np.squeeze(img[:,:,0])
#         green = np.squeeze(img[:,:,1])
#         red = np.squeeze(img[:,:,2])
#         red_transformed = pca.fit_transform(red)
#         #Applying to Green channel and then applying inverse transform to transformed array.
#         green_transformed = pca.fit_transform(green)
#         #Applying to Blue channel and then applying inverse transform to transformed array.
#         blue_transformed = pca.fit_transform(blue)
#         img = np.stack((red_transformed, green_transformed, blue_transformed), axis = 2).tolist()
#         pca_x[i] = (img)
#     return pca_x

In [57]:
x, y = create_train_data()
x.shape

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [12:43<00:00, 26.18it/s]


(20000, 300, 20, 3)

In [26]:
with open('x_20.npy', 'rb') as f:
    x = np.load(f)

In [16]:
with open('labels', 'rb') as f:
    y = pickle.load(f)

In [58]:
country_df = pd.DataFrame({'Country': y})
labelencoder = LabelEncoder()
country_df['Country'] = labelencoder.fit_transform(country_df['Country'])
country_df

,Country
0,0
1,0
2,0
3,0
4,0
...,...
19995,9
19996,9
19997,9
19998,9


In [59]:
x[0][0]

array([[-2.02218279e+00, -4.84941189e-01, -2.41022031e+00],
       [-9.31612531e-01, -5.84405166e-01, -3.12575206e-01],
       [ 2.08270487e-01, -2.01310804e-01, -1.78737620e-01],
       [ 1.05329410e-01,  3.87763808e-01, -1.01444098e-01],
       [ 9.78887471e-02, -5.02803127e-02, -3.81730058e-02],
       [-1.46627036e-02,  1.33104733e-01, -6.50532819e-02],
       [-1.71998721e-02, -1.79898547e-01,  2.87050094e-02],
       [ 4.18145208e-02,  2.00201091e-02, -7.77057712e-02],
       [-5.40797810e-02, -7.21349337e-02,  9.55145637e-02],
       [-2.72220738e-04,  1.08720753e-01,  1.33329190e-01],
       [-7.54291534e-02, -1.41987138e-01, -2.51653967e-02],
       [ 7.68809297e-02, -4.90614934e-04, -1.68306446e-01],
       [ 7.89092957e-02, -9.08037558e-02, -1.22106484e-02],
       [ 2.39034213e-02,  3.03164672e-02,  8.60105706e-02],
       [-5.47912851e-02, -1.31611990e-02, -5.63997254e-02],
       [ 4.04125169e-02,  3.37068588e-02, -1.73724475e-02],
       [ 7.01573263e-02,  1.60954765e-02

In [60]:
class MyDataset():
 
    def __init__(self,x,y):
        self.x_train=torch.tensor(x,dtype=torch.float64)
        self.x_train = torch.reshape((self.x_train), (len(self.x_train), 3, 300, 20))
        self.y_train=torch.tensor(y,dtype=torch.long)
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]


In [61]:
y = country_df["Country"]
md = MyDataset(x,y)

In [62]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [63]:
train_loader=DataLoader(
    md,
    batch_size=64, shuffle=True
)

In [161]:

class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,300,20)
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.fc= nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64 * 75 * 5, num_classes))
    def forward(self, input):
        out = self.layer1(input.float())
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out   

In [162]:
model=ConvNet(num_classes=10).to(device)

In [163]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [164]:
num_epochs=10

In [172]:
ckp_path = "best_checkpoint.model"
checkpoint = torch.load(ckp_path)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
#Model training and saving best model

best_accuracy=0.0
train_count = 20000
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        train_accuracy+=int(torch.sum(prediction==labels.data))
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
    
    torch.save(model.state_dict(),'best_checkpoint.model')

Epoch: 0 Train Loss: tensor(0.4923) Train Accuracy: 0.8302
Epoch: 1 Train Loss: tensor(0.4598) Train Accuracy: 0.8406
Epoch: 2 Train Loss: tensor(0.4510) Train Accuracy: 0.8426
Epoch: 3 Train Loss: tensor(0.4466) Train Accuracy: 0.84665
Epoch: 4 Train Loss: tensor(0.4215) Train Accuracy: 0.85095
Epoch: 5 Train Loss: tensor(0.4121) Train Accuracy: 0.85665


In [92]:
x_test,y_test = create_test_data()

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:19<00:00, 25.91it/s]


In [93]:
country_df = pd.DataFrame({'Country': y_test})
print(country_df["Country"].unique())
labelencoder = LabelEncoder()
country_df['Country_label'] = labelencoder.fit_transform(country_df['Country'])
y_test = country_df["Country_label"]

['ARG' 'AUS,' 'GBR' 'ISR' 'JPN' 'KEN' 'PRT' 'RUS' 'USA' 'ZAF']


In [94]:
mdt = MyDataset(x_test, y_test)

C:\Users\snivy\AppData\Local\Temp\ipykernel_10652\4113779628.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  self.x_train=torch.tensor(x,dtype=torch.float64)


In [95]:
test_loader=DataLoader(
    mdt,
    batch_size=500, shuffle=True
)

In [171]:
from sklearn.metrics import confusion_matrix
test_accuracy = 0
recall = 0
precision = 0
test_count = 500
for i, (images,labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())

    outputs=model(images)
    _,prediction=torch.max(outputs.data,1)
    test_accuracy+=int(torch.sum(prediction==labels.data))
    print(precision_recall_fscore_support(labels.data, prediction, average = None, labels = range(10)))
    print(confusion_matrix(labels.data, prediction))
test_accuracy=test_accuracy/test_count
print("Test Accuracay: " + str(test_accuracy))

(array([0.44186047, 0.09375   , 0.22807018, 0.375     , 0.47619048,
       0.53191489, 0.53658537, 0.21052632, 0.1969697 , 0.41304348]), array([0.38, 0.06, 0.26, 0.36, 0.6 , 0.5 , 0.44, 0.24, 0.26, 0.38]), array([0.40860215, 0.07317073, 0.24299065, 0.36734694, 0.53097345,
       0.51546392, 0.48351648, 0.22429907, 0.22413793, 0.39583333]), array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50], dtype=int64))
[[19  4  5  4  2  3  2  4  3  4]
 [ 6  3  9  4  7  2  1  5 12  1]
 [ 2  4 13  5  2  2  3 10  4  5]
 [ 3  2  4 18  2  2  5  3  4  7]
 [ 1  0  2  4 30  1  2  4  4  2]
 [ 1  2  2  0  4 25  1  1  9  5]
 [ 1  1  7  2  2  3 22  8  4  0]
 [ 5  6  8  3  5  2  3 12  5  1]
 [ 3  7  5  4  7  2  2  5 13  2]
 [ 2  3  2  4  2  5  0  5  8 19]]
Test Accuracay: 0.348


[tensor([5, 0, 5, 3, 7, 5, 2, 2, 3, 0, 1, 0, 7, 6, 7, 6, 4, 8, 9, 7, 5, 4, 5, 2,
        3, 5, 9, 9, 5, 4, 3, 6, 4, 9, 2, 2, 8, 9, 7, 0, 2, 3, 3, 5, 8, 5, 2, 4,
        1, 4, 7, 3, 5, 7, 4, 9, 6, 6, 4, 4, 7, 2, 7, 0, 2, 5, 9, 8, 9, 3, 8, 9,
        2, 3, 7, 7, 5, 3, 3, 7, 8, 5, 9, 3, 2, 9, 9, 9, 7, 8, 8, 5, 7, 4, 4, 5,
        0, 0, 9, 3, 2, 9, 2, 7, 2, 2, 9, 6, 4, 9, 9, 3, 3, 2, 7, 9, 9, 3, 6, 0,
        7, 6, 9, 2, 8, 2, 3, 9, 4, 4, 3, 8, 7, 9, 0, 9, 7, 4, 4, 6, 0, 8, 9, 4,
        1, 5, 2, 5, 0, 2, 2, 8, 7, 7, 0, 0, 9, 8, 7, 0, 7, 4, 9, 0, 4, 5, 9, 3,
        9, 4, 8, 4, 9, 9, 4, 5, 4, 5, 4, 2, 1, 9, 9, 0, 8, 2, 3, 3, 8, 6, 9, 8,
        7, 8, 0, 0, 4, 2, 4, 7, 6, 5, 5, 0, 2, 4, 4, 4, 9, 0, 5, 9, 5, 9, 6, 2,
        8, 6, 3, 6, 7, 5, 6, 0, 7, 2, 9, 8, 9, 7, 4, 2, 4, 2, 3, 5, 8, 1, 3, 4,
        8, 3, 3, 2, 4, 9, 9, 4, 4, 1, 4, 2, 4, 9, 3, 4, 0, 7, 5, 2, 4, 2, 6, 3,
        4, 9, 2, 1, 5, 9, 9, 4, 7, 0, 3, 5, 7, 9, 6, 4, 8, 8, 0, 0, 4, 9, 7, 5,
        6, 6, 7, 3, 4, 7, 5, 6, 3, 0, 1